In [1]:
import os
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from scipy.io.wavfile import read
import wave
from scipy.io.wavfile import read
from sklearn.model_selection import train_test_split
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
import sounddevice as sd
from scipy.io.wavfile import write
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import  Conv1D,  GRU, Dropout, Input, Dense, Conv2D, Reshape, MaxPooling2D
import random
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from scipy import linalg
random.seed(111)
fs = 16000
Keyword_path = "/home/avatar/avatar_project/keyword_sound"
Noise_path =  "/home/avatar/avatar_project/background_sound"

Val_path_pos = "/home/avatar/avatar_project/validation_data/positives"
Val_path_neg = "/home/avatar/avatar_project/validation_data/negatives"

2022-11-14 20:04:42.388232: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 20:04:42.495357: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-14 20:04:42.520309: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-14 20:04:42.969177: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:

#Data augmentation method. New data samples are made by masking parts of mel spectrograms.
def spec_augment(spec: np.ndarray, num_mask=2, 
                 freq_masking_max_percentage=0.10, time_masking_max_percentage=0.2):

    spec = spec.copy()
    for i in range(num_mask):
        all_frames_num, all_freqs_num = spec.shape
        freq_percentage = random.uniform(0.0, freq_masking_max_percentage)
        
        num_freqs_to_mask = int(freq_percentage * all_freqs_num)
        f0 = np.random.uniform(low=0.0, high=all_freqs_num - num_freqs_to_mask)
        f0 = int(f0)
        spec[:, f0:f0 + num_freqs_to_mask] = 0

        time_percentage = random.uniform(0.0, time_masking_max_percentage)
        
        num_frames_to_mask = int(time_percentage * all_frames_num)
        t0 = np.random.uniform(low=0.0, high=all_frames_num - num_frames_to_mask)
        t0 = int(t0)
        spec[t0:t0 + num_frames_to_mask, :] = 0
    
    return spec


In [3]:
keyword_data = []
noise_data = []
test_pos = []
test_neg = []
val_pos = []
val_neg = []
sample_rate = 16000
keyword_data_aug = []
noise_data_aug = []
keyword_audio = []
noise_audio = []
counter = 0
for keyword in os.listdir(Keyword_path):
    audio, fs = librosa.load(os.path.join(Keyword_path, keyword), sr=16000)
    keyword_audio.append(audio)
    
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
    keyword_data.append(mel_spec_db_norm) 
    
    mel_spec_db = librosa.power_to_db(spec_augment(mel_spec), ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
    keyword_data_aug.append(mel_spec_db_norm) 


    
for keyword in os.listdir(Val_path_pos):
    audio, fs = librosa.load(os.path.join(Val_path_pos, keyword), sr=16000)
    
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
    val_pos.append(mel_spec_db_norm) 
    
    
    

for noise in os.listdir(Noise_path):
    audio, fs = librosa.load(os.path.join(Noise_path, noise), sr=16000)
    noise_audio.append(audio)
    
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
    noise_data.append(mel_spec_db_norm)
    
    mel_spec_db = librosa.power_to_db(spec_augment(mel_spec), ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
    noise_data_aug.append(mel_spec_db_norm) 
 


    
for noise in os.listdir(Val_path_neg):
    audio, fs = librosa.load(os.path.join(Val_path_neg, noise), sr=16000)
    
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
    val_neg.append(mel_spec_db_norm) 

In [4]:

#Another augmentation method, where keyword audio is added on top of another keyword audio, and
# same thing is done with noise audio. Audios amplitudes are 
def time_shift(audio): 
    return np.roll(audio,int(sample_rate/(random.randint(3,10)))) #randint means that roll is done with freqs 1600 and 3200

#3 ja 6

for audio_first in keyword_audio[::8]:
    for audio_second in keyword_audio[::8]: 
        audio_shifted = np.add(audio_first*random.uniform(0.8, 1), time_shift(audio_second)*random.uniform(0.4, 0.75))
        
        mel_spec = librosa.feature.melspectrogram(y=audio_shifted, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
        mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
        keyword_data.append(mel_spec_db_norm) 
        
for audio_first in noise_audio[::8]:
    for audio_second in noise_audio[::8]: 
        audio_shifted = np.add(audio_first*random.uniform(0.8, 1), time_shift(audio_second)*random.uniform(0.4, 0.75))
        
        mel_spec = librosa.feature.melspectrogram(y=audio_shifted, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
        mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
        noise_data.append(mel_spec_db_norm)
        

keyword_data = keyword_data + keyword_data_aug
noise_data = noise_data + noise_data_aug

In [ ]:
import librosa.display
fig, ax = plt.subplots()
x = ["","0", "1", "2", "3"]

img = librosa.display.specshow(keyword_data[6], x_axis='time',
y_axis='mel', sr=16000, fmax=8000, ax=ax)

fig.colorbar(img, ax=ax, format='%+2.0f dB')
ax.set_xticklabels(x)
ax.set(title='Augmented Mel-frequency Spectrogram')
plt.savefig('augmented_mel_spec.png')

In [6]:
#Validation and training data split
print(np.shape(keyword_data))
print(np.shape(noise_data))
print(np.shape(val_pos))
print(np.shape(val_neg))

keyword_data = np.squeeze(keyword_data)
noise_data = np.squeeze(noise_data)
val_pos = np.squeeze(val_pos)
val_neg = np.squeeze(val_neg)

keyword_labels = np.ones(keyword_data.shape[0])
val_pos_labels = np.ones(val_pos.shape[0])

noise_labels = 0*np.ones(noise_data.shape[0])
val_neg_labels = 0*np.ones(val_neg.shape[0])

X_train = np.concatenate((keyword_data, noise_data), axis=0)
y_train  = np.concatenate((keyword_labels, noise_labels), axis=0)

X_val = np.concatenate((val_pos, val_neg), axis=0)
y_val = np.concatenate((val_pos_labels, val_neg_labels), axis=0)

y_train = tf.keras.utils.to_categorical(y_train, 2)
y_val = tf.keras.utils.to_categorical(y_val, 2)

(4421, 301, 48)
(4421, 301, 48)
(190, 301, 48)
(190, 301, 48)


In [7]:
from keras.backend import manual_variable_initialization 
manual_variable_initialization(True)

num_units = 48
dropout_ratio = 0.3
#ei ainakaa 0.2, jos 0.3 ei toimi niin viel 0.1
checkpoint_filepath = '/home/avatar/avatar_project/keyword_model_1D_CRNN'
checkpoint_dir = os.path.dirname(checkpoint_filepath)

my_callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=50, mode='max'),
                tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=False, 
                monitor='val_accuracy', mode='max', save_best_only=True),
]

model = Sequential([
    Input(shape=(keyword_data.shape[1], keyword_data.shape[2])),
    Conv1D(num_units,3,padding="same",activation='relu',name='layer1'),
    Dropout(dropout_ratio),
    Conv1D(num_units,3,padding="same",activation='relu',name='layer2'),
    Dropout(dropout_ratio),
    GRU(num_units,name='RNN_1',return_sequences=False),
    Dense(2, activation='softmax', name='dense_a'),
])

model.compile(loss='BinaryCrossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   


2022-11-14 20:06:07.587795: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-14 20:06:07.941196: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9791 MB memory:  -> device: 0, name: NVIDIA TITAN V, pci bus id: 0000:65:00.0, compute capability: 7.0


 layer1 (Conv1D)             (None, 301, 48)           6960      
                                                                 
 dropout (Dropout)           (None, 301, 48)           0         
                                                                 
 layer2 (Conv1D)             (None, 301, 48)           6960      
                                                                 
 dropout_1 (Dropout)         (None, 301, 48)           0         
                                                                 
 RNN_1 (GRU)                 (None, 48)                14112     
                                                                 
 dense_a (Dense)             (None, 2)                 98        
                                                                 
Total params: 28,130
Trainable params: 28,130
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, callbacks=my_callbacks, 
                    validation_data=(X_val, y_val))

Epoch 1/1000


2022-11-14 20:06:10.221382: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2022-11-14 20:06:10.503102: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-14 20:06:10.504200: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-14 20:06:10.504212: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-14 20:06:10.505168: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-14 20:06:10.505209: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


277/277 [==============================] - ETA: 0s - loss: 0.6540 - accuracy: 0.6192

INFO:tensorflow:Assets written to: /home/avatar/avatar_project/keyword_model_1D_CRNN/assets


INFO:tensorflow:Assets written to: /home/avatar/avatar_project/keyword_model_1D_CRNN/assets


...

INFO:tensorflow:Assets written to: /home/avatar/avatar_project/keyword_model_1D_CRNN/assets


277/277 [==============================] - 9s 32ms/step - loss: 0.0246 - accuracy: 0.9904 - val_loss: 0.6623 - val_accuracy: 0.8632
Epoch 120/1000
277/277 [==============================] - 6s 21ms/step - loss: 0.0235 - accuracy: 0.9911 - val_loss: 0.9251 - val_accuracy: 0.8474
Epoch 121/1000
277/277 [==============================] - 6s 22ms/step - loss: 0.0315 - accuracy: 0.9879 - val_loss: 1.0720 - val_accuracy: 0.8000
Epoch 122/1000
277/277 [==============================] - 6s 23ms/step - loss: 0.0209 - accuracy: 0.9924 - val_loss: 0.7990 - val_accuracy: 0.8579
Epoch 123/1000
277/277 [==============================] - 6s 23ms/step - loss: 0.0253 - accuracy: 0.9908 - val_loss: 0.9617 - val_accuracy: 0.8105
Epoch 124/1000
277/277 [==============================] - 6s 22ms/step - loss: 0.0185 - accuracy: 0.9924 - val_loss: 1.0332 - val_accuracy: 0.8395
Epoch 125/1000
277/277 [==============================] - 6s 22ms/step - loss: 0.0242 - accuracy: 0.9906 - val_loss: 1.0462 - val_acc

INFO:tensorflow:Assets written to: /home/avatar/avatar_project/keyword_model_1D_CRNN/assets


INFO:tensorflow:Assets written to: /home/avatar/avatar_project/keyword_model_1D_CRNN/assets


277/277 [==============================] - 9s 31ms/step - loss: 0.0238 - accuracy: 0.9911 - val_loss: 0.7845 - val_accuracy: 0.8711
Epoch 127/1000
277/277 [==============================] - 6s 22ms/step - loss: 0.0241 - accuracy: 0.9911 - val_loss: 0.9088 - val_accuracy: 0.8053
Epoch 128/1000
 49/277 [====>.........................] - ETA: 5s - loss: 0.0274 - accuracy: 0.9885

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model = tf.keras.Model()
#model.load_model(latest)
model=tf.keras.models.load_model('/home/avatar/avatar_project/keyword_model_1D_CRNN')

model.evaluate(X_val, y_val, batch_size=32)

In [ ]:



Test_path_pos = "/home/avatar/avatar_project/test_data/positives/test_reetta_positives"
Test_path_neg = "/home/avatar/avatar_project/test_data/negatives/test_reetta_negatives"
test_pos = []
test_neg = []

for keyword in os.listdir(Test_path_pos):
    audio, fs = librosa.load(os.path.join(Test_path_pos, keyword), sr=16000)
    
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
    test_pos.append(mel_spec_db_norm) 


for noise in os.listdir(Test_path_neg):
    audio, fs = librosa.load(os.path.join(Test_path_neg, noise), sr=16000)
    
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
    test_neg.append(mel_spec_db_norm) 
   

In [ ]:
print(np.shape(test_pos))
print(np.shape(test_neg))
test_pos = np.squeeze(test_pos)
test_neg = np.squeeze(test_neg)
test_pos_labels = np.ones(test_pos.shape[0])
test_neg_labels = 0*np.ones(test_neg.shape[0])

X_test = np.concatenate((test_pos, test_neg), axis=0)
y_test = np.concatenate((test_pos_labels, test_neg_labels), axis=0)
y_test = tf.keras.utils.to_categorical(y_test, 2)

In [ ]:

model.evaluate(X_test, y_test, batch_size=32)
hypothesis= model.predict(X_test, batch_size=32)
y_pred = tf.keras.utils.to_categorical(hypothesis, 2)
from sklearn.metrics import confusion_matrix
y_true = y_test[:, 1]

y_pred = np.round(hypothesis[:, 1])
CM = confusion_matrix(y_true, y_pred)
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
Precision = TP/(TP+FP)
Recall = TP/(TP+FN)
F1_score = 2*((Precision*Recall)/(Precision+Recall))
print(f"true negatives: {TN}, false negatives: {FN}, true positives {TP}, false positives {FP}")
print(f" Precision: {Precision}")
print(f" Recall: {Recall}")
print(f" F1 score: {F1_score}")

In [ ]:
Test_path_pos = "/home/avatar/avatar_project/test_data/positives/test_topias_positives"
Test_path_neg = "/home/avatar/avatar_project/test_data/negatives/test_topias_negatives"
test_pos = []
test_neg = []
for keyword in os.listdir(Test_path_pos):
    audio, fs = librosa.load(os.path.join(Test_path_pos, keyword), sr=16000)
    
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
    test_pos.append(mel_spec_db_norm) 

for noise in os.listdir(Test_path_neg):
    audio, fs = librosa.load(os.path.join(Test_path_neg, noise), sr=16000)
    
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))
    test_neg.append(mel_spec_db_norm) 

In [ ]:
print(np.shape(test_pos))
print(np.shape(test_neg))
test_pos = np.squeeze(test_pos)
test_neg = np.squeeze(test_neg)
test_pos_labels = np.ones(test_pos.shape[0])
test_neg_labels = 0*np.ones(test_neg.shape[0])

X_test = np.concatenate((test_pos, test_neg), axis=0)
y_test = np.concatenate((test_pos_labels, test_neg_labels), axis=0)
y_test = tf.keras.utils.to_categorical(y_test, 2)

In [ ]:
model.evaluate(X_test, y_test, batch_size=32)
hypothesis= model.predict(X_test, batch_size=32)
y_pred = tf.keras.utils.to_categorical(hypothesis, 2)
from sklearn.metrics import confusion_matrix
y_true = y_test[:, 1]



y_pred = np.round(hypothesis[:, 1])
CM = confusion_matrix(y_true, y_pred)
TN = CM[0][0]
FN = CM[1][0]
TP = CM[1][1]
FP = CM[0][1]
Precision = TP/(TP+FP)
Recall = TP/(TP+FN)
F1_score = 2*((Precision*Recall)/(Precision+Recall))
print(f"true negatives: {TN}, false negatives: {FN}, true positives {TP}, false positives {FP}")
print(f" Precision: {Precision}")
print(f" Recall: {Recall}")
print(f" F1 score: {F1_score}")

In [ ]:

sample_rate = 16000
seconds = 3
case=0
recording = sd.rec(int(seconds * fs), samplerate=sample_rate, channels=1)
sd.wait()
first_audio = np.squeeze(recording)
#Checks if wake word has been detected in past 3 seconds
long_audio=first_audio
while True:
    if case == 150:
        break
    case+= 1
    print("Say Now: ")
    recording = sd.rec(int(1 * fs), samplerate=fs, channels=1)
    sd.wait()
    
    audio = np.squeeze(recording)
    long_audio = np.delete(long_audio , [index for index in range(int(np.shape(audio)[0]))] )
    long_audio = np.hstack((long_audio, audio))
    mel_spec = librosa.feature.melspectrogram(y=long_audio, sr=sample_rate, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
    mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db))

    prediction = model.predict(np.expand_dims(mel_spec_db_norm, axis=0), batch_size=32)
    print(prediction)
    if prediction[:, 1] > 0.80:
        print(f"Wake Word Detected for ({case})")
        print("Confidence:", prediction[:, 1])
        #write(test_file + "positive" + str(case) + ".wav", fs, long_audio)
    
    else:
        print(f"Wake Word NOT Detected")
        print("Confidence:", prediction[:, 0])
        #write(test_file + "negative" + str(case) + ".wav", fs, long_audio)
